In [ ]:
import random
from functools import lru_cache
from statistics import mean, median
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from enum import IntEnum
from operator import itemgetter
from copy import copy


POPULATION_SIZE = 1000
MUTATE_PROBABILITY = 0.05
FERTILITY_RATE = 10
TARGET_STRING = 'hello, world'


class Individual():
    '''個体に関する定数クラス
    
    ・遺伝情報
      ・長さ84のビット列で表現する
      ・それぞれ以下に発現する
        ・length(4bit)：文字列長（0~15）
        ・string(5bit * 16)：以下文字を16個
          ・スペース
          ・記号（@!,.?）
          ・小文字アルファベット（a-z）
    '''
    
    fitness = 0
    body = 1
    gene = 2
    
    max_len_str = 16
    gene_size = 84
    expression = ' !,.?@abcdefghijklmnopqrstuvwxyz'
    
    mask_uint4 = 0b1111
    shift_uint4 = 4
    mask_char5 = 0b11111
    shift_char5 = 5


def express(gene=None):
    '''遺伝子から個体を発現する
    
    ・遺伝情報
      ・下位ビットから順に格納されているとする
    '''
    
    if gene is None:
        gene = random.getrandbits(Individual.gene_size)
    else:
        assert 0 <= gene < 2 ** Individual.gene_size
    
    # 遺伝情報を読み解く
    t_gene = gene
    
    t_info_length = (t_gene & Individual.mask_uint4) + 1
    t_gene = t_gene >> Individual.shift_uint4
    
    t_info_string = [0] * Individual.max_len_str
    for i in range(Individual.max_len_str):
        t_info_string[i] = (t_gene & Individual.mask_char5)
        t_gene = t_gene >> Individual.shift_char5
    
    # 遺伝情報から個体へと発現する
    body = [0] * t_info_length
    
    for i in range(t_info_length):
        body[i] = Individual.expression[t_info_string[i]]
    
    body = ''.join(body)
    
    # 適応度を計算する
    fitness = lss(body, TARGET_STRING)
    
    return (fitness, body, gene)


def mate(parent1, parent2):
    '''子供を作る'''
    assert len(parent1) == len(parent2) == 3
    
    t_child_gene = 0
    t_parent1_gene = parent1[Individual.gene]
    t_parent2_gene = parent2[Individual.gene]
    
    # 両親の遺伝子を受け継ぐ
    t_mask_mate = random.getrandbits(Individual.gene_size)
    t_parent1_gene = t_parent1_gene & t_mask_mate
    t_parent2_gene = t_parent2_gene & ~t_mask_mate
    t_child_gene = t_parent1_gene | t_parent2_gene
    
    # 突然変異
    t_mask_mutate = 0
    for _ in range(Individual.gene_size):
        t_mask_mutate = t_mask_mutate << 1
        if random.random() <= MUTATE_PROBABILITY:
            t_mask_mutate = t_mask_mutate | 0b1
    t_child_gene = t_child_gene ^ t_mask_mutate
    
    return express(t_child_gene)


def next_generation(generation=None):
    '''次世代を生み出す'''
    if generation is None:
        t_generation = [express() for _ in range(POPULATION_SIZE)]
    else:
        t_generation = copy(generation)
    
    # エリートと非エリートに分ける
    t_generation.sort(reverse=True)
    t_pareto = POPULATION_SIZE // 5
    t_elites = t_generation[: t_pareto]
    t_non_elites = t_generation[t_pareto :]
    
    # エリートが非エリートと繁殖する
    t_children = []
    for t_parent1 in t_elites:
        while True:
            t_parent2 = random.choice(t_non_elites)
            if t_parent1 is t_parent2:
                continue
            else:
                break
        
        for _ in range(FERTILITY_RATE):
            t_children.append(mate(t_parent1, t_parent2))
    
    # 次世代に生存する個体を選択
    t_elites = random.sample(t_elites, 12 * len(t_elites) // 16)
    t_non_elites = random.sample(t_non_elites, 3 * len(t_non_elites) // 16)
    next_gen = t_elites + t_children + t_non_elites
    next_gen.sort(reverse=True)
    next_gen = next_gen[: POPULATION_SIZE]
    
    return next_gen


def log_generation(generation):
    '''世代の記録を残す'''
    t_max_idx = generation.index(max(generation, key=itemgetter(Individual.fitness)))
    max_body = generation[t_max_idx][Individual.body]
    max_fitness = generation[t_max_idx][Individual.fitness]
    
    min_fitness = min(generation, key=itemgetter(Individual.fitness))[Individual.fitness]
    
    mean_fitness = mean(i[Individual.fitness] for i in generation)
    median_fitness = median(i[Individual.fitness] for i in generation)
    
    return max_body, min_fitness, max_fitness, mean_fitness, median_fitness


@lru_cache(maxsize=4096)
def ld(s, t):
    '''編集距離（レーベンシュタイン距離）を計算する'''
    if not s: return len(t)
    if not t: return len(s)
    if s[0] == t[0]: return ld(s[1:], t[1:])
    l1 = ld(s, t[1:])
    l2 = ld(s[1:], t)
    l3 = ld(s[1:], t[1:])
    return 1 + min(l1, l2, l3)


def lss(s, t):
    '''類似度を計算する（編集距離を標準化し線形変換する）'''
    return -(ld(s, t) / max(len(s), len(t))) + 1


class History(IntEnum):
    '''履歴に関する定数クラス'''
    min = 0
    max = 1
    mean = 2
    median = 3


def hello_world_with_ga():
    '''遺伝的アルゴリズムでhello, world'''
    trials = 1000
    hist = [[], [], [], []]
    gen = next_generation()
    
    for i in range(trials):
        gen = next_generation(gen)
        
        max_body, *history = log_generation(gen)
        print(f'{i} : {max_body}')
        for h in History:
            hist[h].append(history[h])
        if history[History.max] == 1:
            break
    
    x = range(0, len(hist[History.min]))
    plt.figure()
    plt.plot(x, hist[History.min], marker='.', label='min_fitness')
    plt.plot(x, hist[History.max], marker='.', label='max_fitness')
    plt.plot(x, hist[History.mean], marker='.', label='mean_fitness')
    plt.plot(x, hist[History.median], marker='.', label='median_fitness')
    plt.legend(loc='best', fontsize=10)
    plt.grid()
    plt.xlabel('generation')
    plt.ylabel('fitness')
    plt.title(max_body)
    plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
    plt.show()

hello_world_with_ga()